# setup

In [ ]:
# suppress warnings
import warnings
warnings.simplefilter('ignore')

#import packages
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt
import subprocess
from datetime import datetime


#local imports from opensoundscape
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
from opensoundscape.ribbit import ribbit

# create big visuals
plt.rcParams['figure.figsize']=[15,8]
pd.set_option('display.precision', 2)

# read in RIBBIT scores and basic cleaning 

In [157]:
# folder with ribbit scores of interest
ribbit_folder = "ribbit_scores_2020_01_20"

#creates list of all cdv ribbit files to concatonate into one df of all ribbit scores
all_files = glob(ribbit_folder + "/*.csv")
ribbit_list = []

# reads in all ribbit scores to csv 
for filename in all_files: 
    df = pd.read_csv("./" + filename)
    ribbit_list.append(df)
    
ribbit_scores_df = pd.concat(ribbit_list, axis=0, ignore_index=True) # translates list into dataframe 

ribbit_scores_df = ribbit_scores_df.rename(columns={"Unnamed: 0" : "file_path"})

# use file_path to create column with date and time of file #Note (todo):4 files with _Lcapito at end did not get date inputted 
ribbit_scores_df['date'] = pd.to_datetime(test_df.file_path.str[-19:-4], format='%Y%m%d_%H%M%S', errors='coerce')
# ribbit_scores_df.sort_values('date')[-5:-1] # files without dates 

In [159]:
ribbit_scores_df

,file_path,score,time_stamp,year,pond,date
0,/Volumes/Expansion/Frog Call Project/Calling D...,152.62,170.5,2018,400,2018-02-16 23:24:00
1,/Volumes/Expansion/Frog Call Project/Calling D...,134.04,242.5,2018,400,2018-02-12 20:20:00
2,/Volumes/Expansion/Frog Call Project/Calling D...,113.83,176.5,2018,400,2018-04-19 01:09:00
3,/Volumes/Expansion/Frog Call Project/Calling D...,98.98,87.5,2018,400,2018-04-17 22:08:00
4,/Volumes/Expansion/Frog Call Project/Calling D...,98.95,216.5,2018,400,2018-02-11 19:19:00
...,...,...,...,...,...,...
32644,/Volumes/Expansion/Frog Call Project/Calling D...,27.57,220.5,2020,398,2020-03-09 00:40:00
32645,/Volumes/Expansion/Frog Call Project/Calling D...,26.95,35.5,2020,398,2020-04-04 02:59:00
32646,/Volumes/Expansion/Frog Call Project/Calling D...,26.55,24.5,2020,398,2020-05-04 01:20:00
32647,/Volumes/Expansion/Frog Call Project/Calling D...,26.05,48.0,2020,398,2020-04-14 19:06:00


In [156]:
test_df = ribbit_scores_df
test_df['date'] = pd.to_datetime(test_df.file_path.str[-19:-4], format='%Y%m%d_%H%M%S', errors='coerce')
# test_df.sort_values('date')[-5:-1] #note 4 files with _Lcapito at end did not get date inputted 

,file_path,score,time_stamp,year,pond,date
14554,/Volumes/Expansion/Frog Call Project/Calling D...,77.68,258.5,2017,400,NaT
14580,/Volumes/Expansion/Frog Call Project/Calling D...,71.91,175.5,2017,400,NaT
14660,/Volumes/Expansion/Frog Call Project/Calling D...,59.80,264.5,2017,400,NaT
14722,/Volumes/Expansion/Frog Call Project/Calling D...,54.54,22.0,2017,400,NaT


In [141]:
test_df['file_path'][:][-19:-4]

32630    /Volumes/Expansion/Frog Call Project/Calling D...
32631    /Volumes/Expansion/Frog Call Project/Calling D...
32632    /Volumes/Expansion/Frog Call Project/Calling D...
32633    /Volumes/Expansion/Frog Call Project/Calling D...
32634    /Volumes/Expansion/Frog Call Project/Calling D...
32635    /Volumes/Expansion/Frog Call Project/Calling D...
32636    /Volumes/Expansion/Frog Call Project/Calling D...
32637    /Volumes/Expansion/Frog Call Project/Calling D...
32638    /Volumes/Expansion/Frog Call Project/Calling D...
32639    /Volumes/Expansion/Frog Call Project/Calling D...
32640    /Volumes/Expansion/Frog Call Project/Calling D...
32641    /Volumes/Expansion/Frog Call Project/Calling D...
32642    /Volumes/Expansion/Frog Call Project/Calling D...
32643    /Volumes/Expansion/Frog Call Project/Calling D...
32644    /Volumes/Expansion/Frog Call Project/Calling D...
Name: file_path, dtype: object

In [126]:
all_files = glob(ribbit_folder + "/*.csv")


In [124]:
#add pond and year columns - didnt' do this when first running RIBBIT for all years (should not have to use again)
#temp_file = all_files[29]
#temp_df = pd.read_csv(temp_file).rename(columns={"Unnamed: 0" : "file_path"}).drop("label", axis=1, errors = "ignore")
#temp_df['year'] = 2020
#temp_df['pond'] = 398
#temp_df.to_csv(temp_file, index = False)
#pd.read_csv(temp_file)